In [1]:
# Import libraries

import praw
import pandas as pd
import datetime
from json import load

In [25]:
# Read config (client id, secret, user agent) and create Reddit object

CONFIG_PATH = 'config.json'

config = load(open(CONFIG_PATH))

reddit = praw.Reddit(
    client_id=config['reddit_client_id'],
    client_secret=config['reddit_client_secret'],
    user_agent=config['reddit_user_agent'],
)

print("Read only?:", "Yes" if reddit.read_only else "No")

Read only?: Yes


In [29]:
# Example (Check 20 hot posts in r/politics and print title, date)

posts = reddit.subreddit("politics").hot(limit=5)

for i, post in enumerate(posts):
    print("Post {}: \n\t Title: {} \n\t Date: {} \n\t Url: {} \n\t Score: {} \n\t Author: {}".format(
        i, post.title, datetime.datetime.fromtimestamp(post.created), post.url, post.score, post.author))

Post 0: 
	 Title: Discussion Thread: House Jan 6 Public Hearings, Day 1 - 06/09/2022 at 8 pm ET 
	 Date: 2022-06-10 00:58:17 
	 Url: https://www.reddit.com/r/politics/comments/v8tbox/discussion_thread_house_jan_6_public_hearings_day/ 
	 Score: 6299 
	 Author: PoliticsModeratorBot
Post 1: 
	 Title: The January 6 hearings showed why it’s reasonable to call Trump a fascist 
	 Date: 2022-06-11 01:37:55 
	 Url: https://www.vox.com/policy-and-politics/2022/6/10/23162442/january-6-committee-hearing-june-10-trump-fascist 
	 Score: 7365 
	 Author: coffeespeaking
Post 2: 
	 Title: Nearly 20M watched Jan. 6 hearing: Nielsen 
	 Date: 2022-06-10 22:41:48 
	 Url: https://thehill.com/blogs/pundits-blog/media/3519284-nearly-20m-watched-jan-6-hearing-nielsen/ 
	 Score: 46011 
	 Author: h2oape
Post 3: 
	 Title: AOC Asked Gaetz, Boebert, and Greene If They Asked for Pardons After January 6 
	 Date: 2022-06-11 05:33:07 
	 Url: https://www.businessinsider.com/aoc-matt-gaetz-lauren-boebert-marjorie-taylor-g

In [58]:
# Example comment body

comment_body = list(post.comments)[0].body
comment_body

"\nAs a reminder, this subreddit [is for civil discussion.](/r/politics/wiki/index#wiki_be_civil)\n\nIn general, be courteous to others. Debate/discuss/argue the merits of ideas, don't attack people. Personal insults, shill or troll accusations, hate speech, any suggestion or support of harm, violence, or death, and other rule violations can result in a permanent ban. \n\nIf you see comments in violation of our rules, please report them.\n\n For those who have questions regarding any media outlets being posted on this subreddit, please click [here](https://www.reddit.com/r/politics/wiki/approveddomainslist) to review our details as to our approved domains list and outlet criteria.\n \n **Special announcement:**\n \n r/politics is currently accepting new moderator applications.  If you want to help make this community a better place, consider [applying here today](https://www.reddit.com/r/politics/comments/sskg6a/rpolitics_is_looking_for_more_moderators/)!\n\n***\n\n\n*I am a bot, and t